In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')

# Spark SQLContext

## Documentation: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html

Initilizing SQLContext

In [4]:
!rm -rf metastore_db/
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

## Creating user-defined functions

To use RDD in SQLContext, RDD lines have to be converted into a Row format

In [5]:
import re
from pyspark.sql import Row
# Read from CSV
def load_csv(line):
    return re.split("\"?,\"?", line)
        
def readInt(x):
    try:
        return int(x)
    except:
        return x

def parseElement(e):
    return Row(year=readInt(e[0]),
    month=readInt(e[1]),
    day=readInt(e[2]),
    dep_time=readInt(e[3]),
    dep_delay=readInt(e[4]),
    arr_time=readInt(e[5]),
    arr_delay=readInt(e[6]),
    cancelled=e[7],
    carrier=e[8],
    tailnum=e[9],
    flight=readInt(e[10]),
    origin=e[11],
    dest=e[12],
    air_time=readInt(e[13]),
    distance=readInt(e[14]),
    hour=readInt(e[15]),
    min=readInt(e[16])
    )

In [6]:
#!wget https://dsr-data.s3.amazonaws.com/flights/flights14.csv

In [7]:
flights = sc.textFile("flights14.csv").map(load_csv).filter(lambda e: not e[0] == "year").map(parseElement).cache()
#flights = sc.textFile("f1dhhjnzjwxmy2c1ys2whvrgz05d1pui.csv").map(load_csv).filter(lambda e: not e[0] == "year").map(parseElement).cache()

Now the schema and the RDD have to be registered with the sqlContext:

In [8]:
flightsDF = sqlContext.createDataFrame(flights)

In [9]:
flightsDF.createOrReplaceTempView("flight")

In [10]:
sqlContext.sql("select * from flight where dest = 'LAX'").take(5)

[Row(air_time=359, arr_delay=13, arr_time=1238, cancelled='0', carrier='AA', day=1, dep_delay=14, dep_time=914, dest='LAX', distance=2475, flight=1, hour=9, min=14, month=1, origin='JFK', tailnum='N338AA', year=2014),
 Row(air_time=363, arr_delay=13, arr_time=1523, cancelled='0', carrier='AA', day=1, dep_delay=-3, dep_time=1157, dest='LAX', distance=2475, flight=3, hour=11, min=57, month=1, origin='JFK', tailnum='N335AA', year=2014),
 Row(air_time=351, arr_delay=9, arr_time=2224, cancelled='0', carrier='AA', day=1, dep_delay=2, dep_time=1902, dest='LAX', distance=2475, flight=21, hour=19, min=2, month=1, origin='JFK', tailnum='N327AA', year=2014),
 Row(air_time=350, arr_delay=1, arr_time=1706, cancelled='0', carrier='AA', day=1, dep_delay=2, dep_time=1347, dest='LAX', distance=2475, flight=117, hour=13, min=47, month=1, origin='JFK', tailnum='N319AA', year=2014),
 Row(air_time=339, arr_delay=0, arr_time=2145, cancelled='0', carrier='AA', day=1, dep_delay=4, dep_time=1824, dest='LAX', d

In [15]:
flightsDF.show(5)
print(flightsDF.columns)

+--------+---------+--------+---------+-------+---+---------+--------+----+--------+------+----+---+-----+------+-------+----+
|air_time|arr_delay|arr_time|cancelled|carrier|day|dep_delay|dep_time|dest|distance|flight|hour|min|month|origin|tailnum|year|
+--------+---------+--------+---------+-------+---+---------+--------+----+--------+------+----+---+-----+------+-------+----+
|     359|       13|    1238|        0|     AA|  1|       14|     914| LAX|    2475|     1|   9| 14|    1|   JFK| N338AA|2014|
|     363|       13|    1523|        0|     AA|  1|       -3|    1157| LAX|    2475|     3|  11| 57|    1|   JFK| N335AA|2014|
|     351|        9|    2224|        0|     AA|  1|        2|    1902| LAX|    2475|    21|  19|  2|    1|   JFK| N327AA|2014|
|     157|      -26|    1014|        0|     AA|  1|       -8|     722| PBI|    1035|    29|   7| 22|    1|   LGA| N3EHAA|2014|
|     350|        1|    1706|        0|     AA|  1|        2|    1347| LAX|    2475|   117|  13| 47|    1|   JF

In [11]:
flightsDF.where("origin = 'JFK' AND dest = 'MIA'").count()

2750

In [12]:
flightsDF.where("origin = 'JFK' AND dest = 'MIA'").limit(2).collect()

[Row(air_time=161, arr_delay=-17, arr_time=1828, cancelled='0', carrier='AA', day=1, dep_delay=-1, dep_time=1509, dest='MIA', distance=1089, flight=145, hour=15, min=9, month=1, origin='JFK', tailnum='N5FJAA', year=2014),
 Row(air_time=166, arr_delay=-8, arr_time=1227, cancelled='0', carrier='AA', day=1, dep_delay=7, dep_time=917, dest='MIA', distance=1089, flight=1085, hour=9, min=17, month=1, origin='JFK', tailnum='N5DWAA', year=2014)]

### Count the flights that departed early and arrived late

### Find the flightwith the longest arrival delay

### Find the top 10 destinations ordered by the number of flights

### Find top 10 destinations with the worst avg arrival delay, ignoring flights that arrived early

### Take a sample of 1% of the flights and then calculate the average departure delay for that sample

### For all flights from JFK during June, show the average departure delay for each destination

### For every origin/dest pair, count the number of flights